In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Inicjalizacja SparkSession
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

In [0]:
# Zbiór danych - Recenzje filmowe (50 próbek)
data = [
    ("I love this movie, it was fantastic!", 1),
    ("Absolutely terrible, I hated it.", 0),
    ("Best film I have seen in years.", 1),
    ("Not worth the time, very disappointing.", 0),
    ("A wonderful experience, highly recommend!", 1),
    ("The worst film ever, I regret watching it.", 0),
    ("An excellent story with great acting.", 1),
    ("Boring and unoriginal, not worth it.", 0),
    ("I enjoyed every moment, truly inspiring.", 1),
    ("Disastrous, a complete waste of time.", 0),
    ("Pretty decent film, had some great moments.", 1),
    ("Not my cup of tea, felt dull.", 0),
    ("An amazing journey, loved the characters.", 1),
    ("Horrible plot, very predictable.", 0),
    ("Outstanding performance by the cast!", 1),
    ("Extremely dull and uninspiring.", 0),
    ("Brilliantly directed and beautifully shot.", 1),
    ("Mediocre and forgettable.", 0),
    ("Thrilling and full of surprises!", 1),
    ("A pointless sequel, didn't enjoy it.", 0),
    ("Heartwarming and truly moving.", 1),
    ("A disaster of a movie, terrible pacing.", 0),
    ("Beautifully crafted, an absolute gem.", 1),
    ("Unbearably bad, I couldn't finish it.", 0),
    ("Engaging and emotionally satisfying.", 1),
    ("An insult to the original, very poor.", 0),
    ("Absolutely delightful, a must-watch!", 1),
    ("Dreadful and poorly executed.", 0),
    ("Captivating and unforgettable.", 1),
    ("One of the worst movies I've seen.", 0),
    ("A masterpiece, truly exceptional.", 1),
    ("Too slow and boring for my taste.", 0),
    ("Superb acting and a great storyline.", 1),
    ("A shallow and poorly written script.", 0),
    ("Emotionally gripping and powerful.", 1),
    ("Completely overrated and dull.", 0),
    ("Inspirational and beautifully told.", 1),
    ("Amateurish and not worth the watch.", 0),
    ("A spectacular visual treat!", 1),
    ("Ridiculously bad, avoid at all costs.", 0),
    ("An engaging and well-paced thriller.", 1),
    ("Unoriginal and poorly made.", 0),
    ("Loved the humor and the clever script.", 1),
    ("Awful special effects and weak acting.", 0),
    ("A cinematic triumph, pure joy.", 1),
    ("Tedious and lacking any charm.", 0),
    ("Fascinating and thoroughly enjoyable.", 1),
    ("Terrible dialogue and bad pacing.", 0)
]

df = spark.createDataFrame(data, ["text", "label"])

In [0]:
# Tokenizacja
tokenizer = Tokenizer(inputCol="text", outputCol="words")
# Usuwanie stop words
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# TF-IDF
hashing_tf = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1000)
idf = IDF(inputCol="raw_features", outputCol="features")

# Modele klasyfikacyjne
lr = LogisticRegression(featuresCol="features", labelCol="label")
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")
nb = NaiveBayes(featuresCol="features", labelCol="label")

# Tworzenie pipeline
pipeline_lr = Pipeline(stages=[tokenizer, stopwords_remover, hashing_tf, idf, lr])
pipeline_dt = Pipeline(stages=[tokenizer, stopwords_remover, hashing_tf, idf, dt])
pipeline_nb = Pipeline(stages=[tokenizer, stopwords_remover, hashing_tf, idf, nb])

# Podział na zbiór treningowy i testowy
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

In [0]:
print(f"Liczba próbek w zbiorze testowym: {test_df.count()}")
if test_df.count() == 0:
    print("Błąd: zbiór testowy jest pusty! Zwiększ liczbę danych.")
else:
    # Trenowanie modeli
    model_lr = pipeline_lr.fit(train_df)
    model_dt = pipeline_dt.fit(train_df)
    model_nb = pipeline_nb.fit(train_df)

    # Predykcje
    test_predictions_lr = model_lr.transform(test_df)
    test_predictions_dt = model_dt.transform(test_df)
    test_predictions_nb = model_nb.transform(test_df)

    print("Predykcje dla regresji logistycznej:")
    test_predictions_lr.select("text", "label", "prediction").show()
    print("Predykcje dla drzewa decyzyjnego:")
    test_predictions_dt.select("text", "label", "prediction").show()
    print("Predykcje dla Naive Bayes:")
    test_predictions_nb.select("text", "label", "prediction").show()

    evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
    accuracy_lr = evaluator.evaluate(test_predictions_lr)
    accuracy_dt = evaluator.evaluate(test_predictions_dt)
    accuracy_nb = evaluator.evaluate(test_predictions_nb)
    print(f"Accuracy - Logistic Regression: {accuracy_lr:.2f}")
    print(f"Accuracy - Decision Tree: {accuracy_dt:.2f}")
    print(f"Accuracy - Naive Bayes: {accuracy_nb:.2f}")

    # Precision, Recall, F1-score
    evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", metricName="weightedPrecision")
    evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", metricName="weightedRecall")
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", metricName="f1")
    
    print(f"Precision - Logistic Regression: {evaluator_precision.evaluate(test_predictions_lr):.2f}")
    print(f"Recall - Logistic Regression: {evaluator_recall.evaluate(test_predictions_lr):.2f}")
    print(f"F1 Score - Logistic Regression: {evaluator_f1.evaluate(test_predictions_lr):.2f}")
    
    # Analiza wag cech dla regresji logistycznej
    lr_model = model_lr.stages[-1]
    if hasattr(lr_model, "coefficients"):
        feature_importance = sorted(zip(range(len(lr_model.coefficients.toArray())), lr_model.coefficients.toArray()), key=lambda x: abs(x[1]), reverse=True)
        print("Najbardziej wpływowe indeksy cech:")
        print(feature_importance[:10])


Liczba próbek w zbiorze testowym: 16
Predykcje dla regresji logistycznej:
+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|Best film I have ...|    1|       0.0|
|An excellent stor...|    1|       1.0|
|Pretty decent fil...|    1|       1.0|
|An amazing journe...|    1|       1.0|
|Brilliantly direc...|    1|       1.0|
|Horrible plot, ve...|    0|       1.0|
|Mediocre and forg...|    0|       1.0|
|A disaster of a m...|    0|       0.0|
|Beautifully craft...|    1|       1.0|
|A shallow and poo...|    0|       1.0|
|Completely overra...|    0|       0.0|
|Too slow and bori...|    0|       0.0|
|Amateurish and no...|    0|       0.0|
|An engaging and w...|    1|       1.0|
|Ridiculously bad,...|    0|       1.0|
|Fascinating and t...|    1|       1.0|
+--------------------+-----+----------+

Predykcje dla drzewa decyzyjnego:
+--------------------+-----+----------+
|                text|label|prediction|
+----------

1. **Predykcje na zbiorze testowym:**

W zbiorze testowym widać, że regresja logistyczna poprawnie przewidziała etykiety dla niektórych przykładów, ale popełniła również błędy. Przykładowo:

    "Mediocre and forgettable" miało etykietę prawdziwą 0 (negatywną), ale zostało przewidziane jako 1 (pozytywne).
    "Ridiculously bad, avoid at all costs" również zostało błędnie sklasyfikowane jako 1 zamiast 0.

Błędy te wskazują, że model może mieć trudności z rozpoznawaniem negatywnych opinii, szczególnie jeśli są one wyrażone w sposób mniej dosadny lub jeśli zawierają słowa mogące sugerować pozytywny kontekst.

2.** Metryki dla regresji logistycznej:**

    Accuracy (dokładność): 0.69
    Oznacza, że model poprawnie sklasyfikował 69% próbek testowych.
    Precision (precyzja): 0.72
    Spośród wszystkich przykładów przewidzianych jako pozytywne (1), 72% było poprawnych.
    Precyzja sugeruje, że model unika fałszywych alarmów (błędnych przewidywań pozytywnych).
    Recall (czułość): 0.69
    Spośród wszystkich rzeczywistych przykładów pozytywnych, model poprawnie sklasyfikował 69%.
    Niższa czułość oznacza, że model mógł pominąć część pozytywnych próbek.
    F1 Score: 0.68
    F1-score jest zbalansowaną miarą, która uwzględnia zarówno precyzję, jak i czułość. Wynik wskazuje, że model ma średnią skuteczność w klasyfikacji pozytywnych przykładów.

3. **Analiza wpływu cech (ważność cech):**

Najbardziej wpływowe indeksy cech dla regresji logistycznej to:

    Cechy o najwyższej wartości dodatniej:
        Indeks 587, 687, 335, 893, 720, 897, 52, 627 (wartości w przedziale ~3.0–3.2)
        Te cechy (prawdopodobnie odpowiadające specyficznym słowom w tekście) mają największy pozytywny wpływ na przewidywanie klasy 1 (pozytywne opinie).
        Interpretacja: Model traktuje te cechy jako silne wskaźniki pozytywnego sentymentu.

    Cechy o najwyższej wartości ujemnej:
        Indeks 7 (-3.66), 805 (-3.10)
        Te cechy mają największy negatywny wpływ, czyli silnie wskazują na klasę 0 (negatywne opinie).
        Interpretacja: Model postrzega te cechy jako silne wskaźniki negatywnego sentymentu.

4. **Porównanie z innymi modelami:**

    Decision Tree (Accuracy: 0.50):
    Drzewo decyzyjne wykazało się niższą skutecznością. Jego dokładność sugeruje, że działa ono losowo przy tak małym zbiorze testowym. Drzewa decyzyjne mogą być mniej skuteczne w klasyfikacji danych tekstowych bez dostatecznej ilości danych treningowych.

    Naive Bayes (Accuracy: 0.75):
    Model Naive Bayes osiągnął najlepszą dokładność. Jest to zgodne z oczekiwaniami, ponieważ Naive Bayes często sprawdza się w zadaniach klasyfikacji tekstu, szczególnie przy małych zbiorach danych.

5. **Ogólna interpretacja:**

Regresja logistyczna radzi sobie dobrze, ale ma trudności z poprawnym klasyfikowaniem niektórych próbek negatywnych, co wpływa na precyzję i czułość. Warto zwrócić uwagę na najbardziej wpływowe cechy i upewnić się, że dane wejściowe są wystarczająco bogate w reprezentatywne słowa dla obu klas (pozytywnych i negatywnych).